In [3]:
'''
Function Description：使用现有的VGG16搭建一个神经网络以实现乳腺癌分子分型的分类
'''
import tensorflow as tf 
import os
import numpy as np 
#加载训练数据
#以下是train和test的图片集合
batch_size = 20
learning_rate = 1e-3
class_names = ['Her2', 'luminal_A', 'luminal_B', 'TN']#每个分类的名称

In [4]:
#获取train的文件路径
train_data_dir = '/media/ly/liangyi/分子分型/X线MRI共有/MRI_minbox/train'
validation_data_dir = '/media/ly/liangyi/分子分型/X线MRI共有/MRI_minbox/validation'
Her2_train_data_dir = train_data_dir + '/Her_2/'
luminal_A_train_data_dir = train_data_dir + '/luminal_A/'
luminal_B_train_data_dir = train_data_dir + '/luminal_B/'
TN_train_data_dir = train_data_dir + '/TN/'

#将train的图片名依次读取
Her2_train_data_name = [Her2_train_data_dir + filename for filename in os.listdir(Her2_train_data_dir)]
luminal_A_train_data_name = [luminal_A_train_data_dir + filename for filename in os.listdir(luminal_A_train_data_dir)]
luminal_B_train_data_name = [luminal_B_train_data_dir + filename for filename in os.listdir(luminal_B_train_data_dir)]
TN_train_data_name = [TN_train_data_dir + filename for filename in os.listdir(TN_train_data_dir)]

train_data_name = Her2_train_data_name + luminal_A_train_data_name + luminal_B_train_data_name + TN_train_data_name
train_data_name = tf.convert_to_tensor(train_data_name)

#用0,1,2,3分别代表四种类型
train_data_labels = [0]* len(Her2_train_data_name) + [1]*len(luminal_A_train_data_name) + [2]*len(luminal_B_train_data_name) + [3] * len(TN_train_data_name)
train_data_labels = tf.convert_to_tensor(train_data_labels, dtype=tf.int64)
#train_data_name = np.array(train_data_name)
#train_data_labels = np.array(train_data_labels)


In [5]:
print(train_data_name.shape)
print(train_data_labels.shape)

(583,)
(583,)


In [37]:
print(len(Her2_train_data_name), len(luminal_A_train_data_name), len(luminal_B_train_data_name), len(TN_train_data_name))

189 125 148 121


In [6]:
#获取验证图像的文家路径
Her2_val_data_dir = validation_data_dir + '/Her_2/'
luminal_A_val_data_dir = validation_data_dir + '/luminal_A/'
luminal_B_val_data_dir = validation_data_dir + '/luminal_B/'
TN_val_data_dir = validation_data_dir + '/TN/'
#获取验证图像的文件名称
Her2_val_data_name = [Her2_val_data_dir + filename for filename in os.listdir(Her2_val_data_dir)]
luminal_A_val_data_name = [luminal_A_val_data_dir + filename for filename in os.listdir(luminal_A_val_data_dir)]
luminal_B_val_data_name = [luminal_B_val_data_dir + filename for filename in os.listdir(luminal_B_val_data_dir)]
TN_val_data_name = [TN_val_data_dir + filename for filename in os.listdir(TN_val_data_dir)]
val_data_name = Her2_val_data_name + luminal_A_val_data_name + luminal_B_val_data_name + TN_val_data_name

val_data_labels = [0] * len(Her2_val_data_name) + [1] * len(luminal_A_val_data_name) + [2] * len(luminal_B_val_data_name) + [3] * len(TN_val_data_name)
#val_data_name = np.array(val_data_name)
#val_data_labels = np.array(val_data_labels)
val_data_name = tf.convert_to_tensor(val_data_name)
val_data_labels = tf.convert_to_tensor(val_data_labels)

In [7]:
print(val_data_name.shape)
print(val_data_labels.shape)

(250,)
(250,)


In [8]:
#创建映射函数
def _map(imagename, label):
    image_string = tf.io.read_file(imagename)
    image_decoded = tf.image.decode_png(image_string, channels = 1)
    image_resize = tf.image.resize(image_decoded, [28,28])/255.0
    return image_resize, label


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data_name, train_data_labels))


In [10]:
train_dataset = train_dataset.map(
    map_func = _map,
    num_parallel_calls= tf.data.experimental.AUTOTUNE
)

In [11]:
#对训练集合做出设定， 
train_dataset = train_dataset.shuffle(buffer_size = 1000).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [12]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_data_name, val_data_labels))
val_dataset = val_dataset.map(_map).batch(batch_size)

In [13]:
print(train_dataset, val_dataset)# dataset 本身无法方便地获取元素

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)> <BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int32)>


In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation = tf.nn.relu, input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 5, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = tf.nn.relu),
    tf.keras.layers.Dense(4, activation = tf.nn.sigmoid)
])

In [15]:
num_epochs = 30

In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [17]:
model.fit(train_dataset, epochs = num_epochs)

Epoch 1/30
30/30 [==============================] - 0s 3ms/step - loss: 1.3749 - sparse_categorical_accuracy: 0.3087
Epoch 2/30
30/30 [==============================] - 0s 2ms/step - loss: 1.3740 - sparse_categorical_accuracy: 0.3242
Epoch 3/30
30/30 [==============================] - 0s 2ms/step - loss: 1.3699 - sparse_categorical_accuracy: 0.3242
Epoch 4/30
30/30 [==============================] - 0s 2ms/step - loss: 1.3669 - sparse_categorical_accuracy: 0.3242
Epoch 5/30
30/30 [==============================] - 0s 2ms/step - loss: 1.3650 - sparse_categorical_accuracy: 0.3242
Epoch 6/30
30/30 [==============================] - 0s 2ms/step - loss: 1.3663 - sparse_categorical_accuracy: 0.3328
Epoch 7/30
30/30 [==============================] - 0s 2ms/step - loss: 1.3473 - sparse_categorical_accuracy: 0.3293
Epoch 8/30
30/30 [==============================] - 0s 2ms/step - loss: 1.3198 - sparse_categorical_accuracy: 0.3739
Epoch 9/30
30/30 [==============================] - 0s 2ms/step 

In [18]:
model.evaluate(val_dataset)

13/13 [==============================] - 0s 5ms/step - loss: 1.1285 - sparse_categorical_accuracy: 0.5200


[1.128541350364685, 0.5199999809265137]

In [32]:
print(val_dataset)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int32)>


In [33]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation = tf.nn.relu, input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = tf.nn.relu),
    tf.keras.layers.Dense(64, activation = tf.nn.relu),
    tf.keras.layers.Dense(4, activation = tf.nn.sigmoid)
])
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [36]:
model1.fit(train_dataset, validation_data = val_dataset, epochs = 40)

Epoch 1/40
30/30 [==============================] - 0s 5ms/step - loss: 0.7830 - sparse_categorical_accuracy: 0.6518 - val_loss: 1.3824 - val_sparse_categorical_accuracy: 0.4840
Epoch 2/40
30/30 [==============================] - 0s 5ms/step - loss: 0.7997 - sparse_categorical_accuracy: 0.6329 - val_loss: 1.2272 - val_sparse_categorical_accuracy: 0.5320
Epoch 3/40
30/30 [==============================] - 0s 5ms/step - loss: 0.7417 - sparse_categorical_accuracy: 0.6604 - val_loss: 1.3101 - val_sparse_categorical_accuracy: 0.4800
Epoch 4/40
30/30 [==============================] - 0s 5ms/step - loss: 0.7030 - sparse_categorical_accuracy: 0.6861 - val_loss: 1.2978 - val_sparse_categorical_accuracy: 0.5520
Epoch 5/40
30/30 [==============================] - 0s 5ms/step - loss: 0.6495 - sparse_categorical_accuracy: 0.7238 - val_loss: 1.1758 - val_sparse_categorical_accuracy: 0.5640
Epoch 6/40
30/30 [==============================] - 0s 5ms/step - loss: 0.7644 - sparse_categorical_accuracy: 